In [1]:
import pandas as pd
import numpy as np

In [2]:
import json
import os
import pandas as pd

input_directory = './Jsons'
output_file = './mobile_data.csv'

columns = [
    "name", "brand", "2G", "3G", "4G", "5G", "Announced", "Status", "Weight", "Length", "Width", 
    "Diameter", "SIM", "Display Type", "Display Size", "PPI", "Body Ratio", "OS", "Battery_capacity", 
    "Price", "CPU", "Ratio", "Pixel", "WLAN", "Colors", "Sensors", "Bluetooth", "GPU", 
    "LoudSpeaker", "3.5mm jack", "Chipset", "Network", "Internal", "Card slot", "RAM", "Storage"
]
# Initialize an empty list to store rows for the CSV file
data = []

# Function to extract data from each JSON file
def extract_data(file_path, brand_name):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = json.load(file)
        
        for model, specs in content.get(brand_name, {}).items():
            dimensions = specs.get("Body", [{}])[0].get("Dimensions", "")
            dimensions_split = dimensions.split(' x ') if dimensions else []
            price = next((item.get("Price", "") for item in specs.get("Misc", []) if "Price" in item), "")
            chipset = next((item.get("Chipset", "") for item in specs.get("Platform", []) if "Chipset" in item), "")
            gpu = next((item.get("GPU", "") for item in specs.get("Platform", []) if "GPU" in item), "")
            ram = next((item.get("Internal", "") for item in specs.get("Memory", []) if "RAM" in item.get("Internal", "")), "")
            storage = next((item.get("Internal", "") for item in specs.get("Memory", []) if "Internal" in item), "")
            row = {
                "name": model,
                "brand": brand_name,
                "2G": any("2G" in spec.get("Technology", "") for spec in specs.get("Network", [])),
                "3G": any("3G" in spec.get("Technology", "") for spec in specs.get("Network", [])),
                "4G": any("4G" in spec.get("Technology", "") for spec in specs.get("Network", [])),
                "5G": any("5G" in spec.get("Technology", "") for spec in specs.get("Network", [])),
                "Announced": specs.get("Launch", [{}])[0].get("Announced", ""),
                "Status": specs.get("Launch", [{}])[1].get("Status", ""),
                "Weight": specs.get("Body", [{}])[1].get("Weight", ""),
                "Length": dimensions_split[0] if len(dimensions_split) > 0 else "",
                "Width": dimensions_split[1] if len(dimensions_split) > 1 else "",
                "Diameter": dimensions_split[2] if len(dimensions_split) > 2 else "",
                "SIM": specs.get("Body", [{}])[2].get("SIM", ""),
                "Display Type": specs.get("Display", [{}])[0].get("Type", ""),
                "Display Size": specs.get("Display", [{}])[1].get("Size", ""),
                "PPI": specs.get("Display", [{}])[2].get("Resolution", ""),
                "Body Ratio": specs.get("Display", [{}])[1].get("Size", "").split("(~")[-1].replace(" screen-to-body ratio)", "") if "(~" in specs.get("Display", [{}])[1].get("Size", "") else "",
                "OS": specs.get("Platform", [{}])[0].get("OS", ""),
                "Battery_capacity": specs.get("Battery", [{}])[0].get("Type", ""),
                "Price": price,
                "CPU": chipset,
                "Ratio": specs.get("Display", [{}])[2].get("Resolution", ""),
                "Pixel": specs.get("Display", [{}])[2].get("Resolution", "").split("(~")[-1] if "(~" in specs.get("Display", [{}])[2].get("Resolution", "") else "",
                "WLAN": specs.get("Comms", [{}])[0].get("WLAN", ""),
                "Colors": specs.get("Misc", [{}])[0].get("Colors", ""),
                "Sensors": specs.get("Features", [{}])[0].get("Sensors", ""),
                "Bluetooth": specs.get("Comms", [{}])[1].get("Bluetooth", ""),
                "GPU": gpu,
                "LoudSpeaker": specs.get("Sound", [{}])[0].get("Loudspeaker", "") == "Yes",
                "3.5mm jack": specs.get("Sound", [{}])[1].get("3.5mm jack", "") == "Yes",
                "Chipset": chipset,
                "Network": specs.get("Network", [{}])[0].get("Technology", ""),
                "Internal": specs.get("Memory", [{}])[0].get("Internal", ""),
                "Card Slot": specs.get("Memory", [{}])[0].get("Card slot", ""),
                "RAM": ram,
                "Storage": ram
            }
            data.append(row)

# Process each JSON file
for file_name in os.listdir(input_directory):
    if file_name.endswith(".json"):
        brand_name = file_name.split("_")[0]
        extract_data(os.path.join(input_directory, file_name), brand_name)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(data, columns=columns)
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Data has been extracted and saved to {output_file}")


Data has been extracted and saved to ./mobile_data.csv


In [3]:
df

,name,brand,2G,3G,4G,5G,Announced,Status,Weight,Length,...,Bluetooth,GPU,LoudSpeaker,3.5mm jack,Chipset,Network,Internal,Card slot,RAM,Storage
0,Xperia Arc S,Sony,False,False,False,False,"2011, August. Released 2011, September",Discontinued,117 g (4.13 oz),125,...,"2.1, A2DP, EDR",Adreno 205,True,True,Qualcomm MSM8255T Snapdragon S2,GSM / HSPA,,NaN,"1GB (320MB user available), 512MB RAM","1GB (320MB user available), 512MB RAM"
1,Xperia neo V,Sony,False,False,False,False,"2011, August. Released 2011, October",Discontinued,126 g (4.44 oz),116,...,"2.1, A2DP, EDR",Adreno 205,True,True,Qualcomm MSM8255 Snapdragon S2,GSM / HSPA,,NaN,"1GB (320MB user available), 512MB RAM","1GB (320MB user available), 512MB RAM"
2,Live with Walkman,Sony,False,False,False,False,"2011, August. Released 2011, October",Discontinued,115 g (4.06 oz),106,...,"2.1, A2DP, EDR",Adreno 205,False,True,Qualcomm MSM8255 Snapdragon S2,GSM / HSPA,,NaN,320MB 512MB RAM,320MB 512MB RAM
3,Xperia ray,Sony,False,False,False,False,"2011, June. Released 2011, August",Discontinued,100 g (3.53 oz),111,...,"2.1, A2DP, EDR",Adreno 205,True,True,Qualcomm MSM8255 Snapdragon S2,GSM / HSPA,,NaN,"1GB (300MB user available), 512MB RAM","1GB (300MB user available), 512MB RAM"
4,Xperia active,Sony,False,False,False,False,"2011, June. Released 2011, October",Discontinued,110.8 g (3.88 oz),92,...,"2.1, A2DP, EDR",Adreno 205,True,True,Qualcomm MSM8255 Snapdragon S2,GSM / HSPA,,NaN,"1GB (320MB user available), 512MB RAM","1GB (320MB user available), 512MB RAM"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,P800,Sony,False,False,False,False,"2002, September",Discontinued,158 g (5.57 oz),117,...,Yes,,False,False,,GSM,,NaN,,
184,Z700,Sony,False,False,False,False,"2002, September",Cancelled,95 g (3.35 oz),91,...,Yes,,True,False,,GSM,,NaN,,
185,T68i,Sony,False,False,False,False,"2002, April",Discontinued,84 g (2.96 oz),101,...,1.0b,,False,False,,GSM,,NaN,,
186,Xperia Duo,Sony,False,False,False,False,Not officially announced yet,Cancelled,-,-,...,"2.1, A2DP, EDR",Adreno 220,True,True,Qualcomm MSM8260 Snapdragon S3,GSM / HSPA,,NaN,1.5GB RAM,1.5GB RAM
